In [22]:
import json
import gzip
from json.decoder import JSONDecodeError
import boto3
import requests
from pathlib import Path

url = "http://bulk.openweathermap.org/sample/daily_14.json.gz"
tmpFile = "daily_14.json.gz"
htmlFile = "msd.html"
bucket = "aws-public"


# Download file
r = requests.get(url)
filename = Path(tmpFile)
filename.write_bytes(r.content)

# Upload file to S3
s3 = boto3.client('s3')
with open(tmpFile, "rb") as f:
    s3.upload_fileobj(f, bucket, tmpFile)

def load_data(mystr):
    splitted = mystr.split('\n')
    for e in splitted:
        try:
            yield json.loads(e)
        except GeneratorExit:
            raise GeneratorExit
        except:
            pass

with gzip.open("daily_14.json.gz", 'rb') as jsgz:
    json_bytes = jsgz.read()

json_str = json_bytes.decode('utf-8')
data = load_data(json_str)
for line in data:
    if line["city"]["name"] == "Prague":
        clouds = line["data"][0]["clouds"]
        break

htmlclouds = ""
for i in range(int(clouds)): 
    htmlclouds+="<img src=\"cloud-icon-small.png\" />"

html = f"""
<html><body>
<h2>Prague</h2>
<h3>Clouds: {clouds}</j3>
<hr />
{htmlclouds}
</body></html>
"""

with open(htmlFile, "w") as hfile:
    hfile.write(html)

with open(htmlFile, "rb") as f:
    s3.upload_fileobj(f, bucket, htmlFile)